In [11]:
import torch
import numpy as np

In [12]:
data = [[1,2],[3,4]]
x_data = torch.tensor(data)

In [13]:
x_data

tensor([[1, 2],
        [3, 4]])

In [14]:
if torch.cuda.is_available():
  tensor = x_data.to('cuda')
  print(f"Device tensor is stored on: {tensor.device}")

In [18]:
tensor = torch.rand(4,4)
print(tensor.matmul(tensor.T))

tensor([[2.4133, 2.4983, 2.4190, 1.2115],
        [2.4983, 2.7467, 2.5316, 1.3318],
        [2.4190, 2.5316, 2.4428, 1.2047],
        [1.2115, 1.3318, 1.2047, 0.9811]])


In [9]:
import math
import torch

dtype = torch.float
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"

print(device)
torch.set_default_device(device)

x = torch.linspace(-math.pi, math.pi, 2000, dtype=dtype)
y = torch.sin(x)

a = torch.randn((), dtype=dtype, requires_grad=True)
b = torch.randn((), dtype=dtype, requires_grad=True)
c = torch.randn((), dtype=dtype, requires_grad=True)
d = torch.randn((), dtype=dtype, requires_grad=True)

learning_rate = 1e-6
for t in range(2000):
  y_pred = a+b*x+c**x*2+d*x**3
  loss = (y_pred-y).pow(2).sum()
  if t%100 == 99:
    print(t, loss.item())
  loss.backward()

  with torch.no_grad():
    a -= learning_rate * a.grad
    b -= learning_rate * b.grad
    c -= learning_rate * c.grad
    d -= learning_rate * d.grad

    a.grad=None
    b.grad=None
    c.grad=None
    d.grad=None
print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

cpu
99 11718.34765625
199 5882.94775390625
299 3037.03076171875
399 1616.2857666015625
499 889.644775390625
599 509.13671875
699 305.3062438964844
799 193.5824432373047
899 130.76910400390625
999 94.3636245727539
1099 72.4515380859375
1199 58.64185333251953
1299 49.46602249145508
1399 43.01983642578125
1499 38.243160247802734
1599 34.53528594970703
1699 31.547971725463867
1799 29.0725040435791
1899 26.978870391845703
1999 25.182071685791016
Result: y = -2.0981967449188232 + 0.47371718287467957 x + 1.1777191162109375 x^2 + -0.0868775025010109 x^3


In [10]:
#autograd
class LegendrePolynomial3(torch.autograd.Function):
  @staticmethod
  def forward(ctx, input):
    ctx.save_for_backward(input)
    return 0.5*(5*input**3 - 3*input)
  @staticmethod
  def backward(ctx, grad_output):
    input, = ctx.saved_tensors
    return grad_output*1.5*(5*input**2-1)

x = torch.linspace(-math.pi, math.pi, 2000, dtype=dtype)
y = torch.sin(x)

a = torch.full((),0.0, dtype=dtype, requires_grad=True)
b = torch.full((),-1.0, dtype=dtype, requires_grad=True)
c = torch.full((),0.0, dtype=dtype, requires_grad=True)
d = torch.full((),0.3, dtype=dtype, requires_grad=True)

learning_rate = 5e-6

for t in range(2000):
  P3 = LegendrePolynomial3.apply
  y_pred = a+b*P3(c+d*x)
  loss = (y_pred-y).pow(2).sum()
  if t%100 == 99:
    print(t, loss.item())
  loss.backward()
with torch.no_grad():
    a -= learning_rate * a.grad
    b -= learning_rate * b.grad
    c -= learning_rate * c.grad
    d -= learning_rate * d.grad

    a.grad=None
    b.grad=None
    c.grad=None
    d.grad=None
print(f'Result: y = {a.item()} + {b.item()} x + {c.item()} x^2 + {d.item()} x^3')

99 461.902587890625
199 461.902587890625
299 461.902587890625
399 461.902587890625
499 461.902587890625
599 461.902587890625
699 461.902587890625
799 461.902587890625
899 461.902587890625
999 461.902587890625
1099 461.902587890625
1199 461.902587890625
1299 461.902587890625
1399 461.902587890625
1499 461.902587890625
1599 461.902587890625
1699 461.902587890625
1799 461.902587890625
1899 461.902587890625
1999 461.902587890625
Result: y = -4.76837147544984e-09 + -4.203699111938477 x + 2.47955313170678e-07 x^2 + -67.89057159423828 x^3
